In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import numpy as np

embeddings_names = [
    "blood_cell_scrna",
    "lymphoma",
    "cifar_100",
    "mnist",
]
sigs = [
    [(1, 2), (0, 2), (-1, 2), (-1, 2), (-1, 2)],
    [(1, 2), (1, 2)],
    [(1, 2), (1, 2), (1, 2), (1, 2)],
    [(1, 2), (0, 2), (-1, 2)],
]
n_trials = 10
sets = ["train", "test"]
datasets = ["X", "y"]

bad = []
for embedding in embeddings_names:
    for trial in range(n_trials):
        for set_name in sets:
            for dataset in datasets:
                my_data = np.load(f"../data/{embedding}/embeddings/{dataset}_{set_name}_{trial}.npy")
                if np.isnan(my_data).any():
                    bad.append((embedding, trial, set_name, dataset))
                    print(embedding, trial, set_name, dataset)
                # print(my_data.shape)
print(bad)

[]


In [6]:
import embedders
import pandas as pd
from tqdm.notebook import tqdm

# N_SAMPLES = 100 # Takes ~20 secs
# N_SAMPLES = 1_000 # Takes ~5 mins
N_SAMPLES = float("inf")  # Takes ~1 hour
MAX_DEPTH = 3
N_FEATURES = "d_choose_2"

results = []
my_tqdm = tqdm(total=len(embeddings_names) * n_trials)
for embedding, sig in zip(embeddings_names, sigs):
    pm = embedders.manifolds.ProductManifold(signature=sig)
    for trial in range(n_trials):
        X_train = np.load(f"../data/{embedding}/embeddings/X_train_{trial}.npy")
        y_train = np.load(f"../data/{embedding}/embeddings/y_train_{trial}.npy")
        X_test = np.load(f"../data/{embedding}/embeddings/X_test_{trial}.npy")
        y_test = np.load(f"../data/{embedding}/embeddings/y_test_{trial}.npy")

        # Randomly subsample
        if len(X_train) > N_SAMPLES:
            idx = np.random.choice(X_train.shape[0], N_SAMPLES, replace=False)
            X_train = X_train[idx]
            y_train = y_train[idx]

        if len(X_test) > N_SAMPLES:
            idx = np.random.choice(X_test.shape[0], N_SAMPLES, replace=False)
            X_test = X_test[idx]
            y_test = y_test[idx]

        res = embedders.benchmarks.benchmark(
            X=None,
            y=None,
            X_train=X_train,
            X_test=X_test,
            y_train=y_train,
            y_test=y_test,
            pm=pm,
            # models=["sklearn_dt", "product_dt"],
            max_depth=MAX_DEPTH,
            batch_size=1,
            n_features=N_FEATURES,
        )
        res["embedding"] = embedding
        res["trial"] = trial

        results.append(res)
        my_tqdm.update(1)

results = pd.DataFrame(results)

  0%|          | 0/40 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [51]:
results.to_csv("../data/results/vae.tsv", sep="\t", index=False)

In [49]:
results.groupby("embedding").mean()

,sklearn_dt,sklearn_rf,product_dt,product_rf,tangent_dt,tangent_rf,knn,ps_perceptron,trial
embedding,,,,,,,,,
blood_cell_scrna,0.1633,0.1746,0.1541,0.1703,0.1692,0.1732,0.1635,0.0296,4.5
cifar_100,0.0907,0.0977,0.0913,0.1044,0.0898,0.0991,0.0795,0.0546,4.5
lymphoma,0.8192,0.8172,0.8314,0.8291,0.8156,0.8154,0.7872,0.7821,4.5
mnist,0.2668,0.3413,0.2909,0.3873,0.2646,0.3463,0.3386,0.1687,4.5
